In [196]:
import osmnx as ox
import networkx as nx

import geopandas as gpd
import pandas as pd
import numpy as np
from statistics import mean

import matplotlib.pyplot as plt
import shapely
import folium

import os

In [197]:
location = 'Richmond, California, USA'
richmond_center = [37.943882, -122.35342]

ca_teale_albers_nad83 = 'NAD_1983_California_Teale_Albers_FtUS'
nad83 = 'EPSG:4629'

wgs84 = 'EPSG:4326'
wgs84_m = 'EPSG:4798'

In [198]:
def plot_gdf_with_background(gdf, zoom=12, center = None):
    
    if center is not None:
        map_center = center
    else:
        map_center_lon = gdf['geometry'].x.mean()
        map_center_lat = gdf['geometry'].y.mean()
        map_center = [map_center_lat, map_center_lon]
    
    map_with_background = folium.Map(map_center, zoom_start = zoom, width = '50%', height = '50%', 
                                     tiles = 'OpenStreetMap')
    
    folium.GeoJson(gdf).add_to(map_with_background)
    
    return(map_with_background)

## Read in and Clean Necessary Data

In [199]:
import warnings

warnings.filterwarnings('ignore')

graph_raw = ox.graph_from_place(location, network_type = 'drive')

graph = ox.project_graph(graph_raw, to_crs = wgs84)
graph = ox.add_edge_speeds(graph)
graph = ox.add_edge_travel_times(graph)

graph_nodes_gdf, graph_edges_gdf = ox.graph_to_gdfs(graph, nodes = True, edges = True)

graph_edges_gdf_reset = graph_edges_gdf.reset_index()

#ox.plot_graph(graph)

In [200]:
hubs_path = os.path.join(os.getcwd(), 'data', 'candidate_site_campuses_2021-11-17', 'candidate_site_campuses.csv')

hubs_df_raw = pd.read_csv(hubs_path)
hubs_df_raw = hubs_df_raw.loc[hubs_df_raw['cat_site'] != 'X', ['id_site', 'SQFT_ROOF', 'LON', 'LAT']]

hubs_geom = gpd.points_from_xy(hubs_df_raw.LON, hubs_df_raw.LAT, crs = wgs84)
hubs_gdf = gpd.GeoDataFrame(hubs_df_raw, geometry = hubs_geom, crs = wgs84)

#plot_gdf_with_background(hubs_gdf, 13, richmond_center)

In [201]:
cengeos_path = os.path.join(os.getcwd(), 'data', 'bg_ca_19', 'blockgroup_CA_19.shp')

cengeos_gdf = gpd.read_file(cengeos_path)
cengeos_gdf = cengeos_gdf.to_crs(hubs_gdf.crs)

#cengeos_gdf.plot()

In [202]:
cengeos_lons = [float(intpt) for intpt in cengeos_gdf['INTPTLON']]
cengeos_lats = [float(intpt) for intpt in cengeos_gdf['INTPTLAT']]

cengeos = pd.DataFrame(cengeos_gdf[['GISJOIN', 'COUNTYFP']])
cengeos['LON'] = cengeos_lons
cengeos['LAT'] = cengeos_lats

cengeos_pt_geom = gpd.points_from_xy(x = cengeos.LON,y = cengeos.LAT, crs = wgs84)
cengeos_pt_gdf = gpd.GeoDataFrame(cengeos, geometry = cengeos_pt_geom, crs = wgs84)

#plot_gdf_with_background(cengeos_pt_gdf, 13, richmond_center)

## Restrict to Hubs within Desired Area

In [203]:
lon_max = graph_nodes_gdf['lon'].max()
lon_min = graph_nodes_gdf['lon'].min()
lon_avg = (lon_max + lon_min)/2

lat_max = graph_nodes_gdf['lat'].max()
lat_min = graph_nodes_gdf['lat'].min()
lat_avg = (lat_min + lat_max)/2

bbox_coords = [(lon_min, lat_min), (lon_min, lat_max), (lon_max, lat_max), (lon_max, lat_min)]
bbox_poly = shapely.geometry.Polygon(bbox_coords)

In [211]:
hubs_gdf_bbox = hubs_gdf[hubs_gdf.within(bbox_poly)]

#plot_gdf_with_background(hubs_gdf_bbox)

In [212]:
cengeos_pt_gdf_bbox = cengeos_pt_gdf[cengeos_pt_gdf.within(bbox_poly)]

### REMOVE THE FOLLOWING LINE AFTER TESTING!!! - selecting random BGs for matrix
#cengeos_pt_gdf_bbox = cengeos_pt_gdf_bbox.iloc[[0, 3, 12, 20, 48, 65], ]

#plot_gdf_with_background(cengeos_pt_gdf_bbox, center = richmond_center)

## Build Distance Matrix for Desired Area

In [213]:
n_cengeos = len(cengeos_pt_gdf_bbox)
n_hubs = len(hubs_gdf_bbox)

In [214]:
name_index = {i:cengeos_pt_gdf_bbox.iloc[i]['GISJOIN'] for i in range(0, n_cengeos)}
name_columns = {i:hubs_gdf_bbox.iloc[i]['id_site'] for i in range(0, n_hubs)}

dist_to_hub_matrix = np.NaN*np.zeros((len(cengeos_pt_gdf_bbox), len(hubs_gdf_bbox))) #what does np.zeros do?
dist_to_hub_df = pd.DataFrame(dist_to_hub_matrix)

dist_to_hub_df.rename(index = name_index, columns = name_columns, inplace = True)

In [215]:
cengeos_buffer = cengeos_pt_gdf_bbox.buffer(0.025)

In [216]:
for cengeo in range(0, n_cengeos):
    
    id_cengeo = cengeos_pt_gdf_bbox.iloc[cengeo]['GISJOIN']
    #print('Getting nearest hubs for', id_cengeo)
    
    cengeo_coords = [cengeos_pt_gdf_bbox.iloc[cengeo]['geometry'].y, cengeos_pt_gdf_bbox.iloc[cengeo]['geometry'].x]
    node_origin = ox.get_nearest_node(graph, cengeo_coords, method = 'euclidean') #technically should be haversine bc decimal degrees

    cengeo_buffer = cengeos_buffer.iloc[cengeo]
    
    hubs_nearby_gdf = hubs_gdf_bbox[hubs_gdf_bbox.within(cengeo_buffer)]
    n_hubs_nearby = len(hubs_nearby_gdf)
    
    for hub in range(0, n_hubs_nearby):
    
        id_hub = hubs_nearby_gdf.iloc[hub]['id_site']
        #print('Calculating drive time to', id_hub)

        hub_geom = [hubs_nearby_gdf.iloc[hub]['geometry'].y, hubs_nearby_gdf.iloc[hub]['geometry'].x]
        node_target = ox.get_nearest_node(graph, hub_geom, method='euclidean')

        route_bt_nodes = nx.shortest_path(graph, node_origin, node_target, weight='travel_time')

        travel_times_sec = graph_edges_gdf_reset.loc[graph_edges_gdf_reset['v'].isin(route_bt_nodes), 'travel_time']
        travel_time_min = round(travel_times_sec.sum()/60, 2)

        dist_to_hub_df.loc[id_cengeo].iloc[hub] = travel_time_min

In [217]:
dist_to_hub_df

,166435596,166436530,195870835,221319847,222473836,223125608,223289015,223289016,232019756,237881575,...,481599042,481599050,481792976,481794537,481799727,482067952,482068034,484701172,834309464,834763586
G06001303620001,12.84,8.54,13.80,6.47,10.04,1.82,11.52,8.10,13.40,11.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
G06001303620002,9.70,0.06,13.93,8.12,13.65,14.05,15.13,10.40,16.13,16.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
G06001303620003,9.09,12.56,4.41,8.80,5.05,11.34,13.82,7.38,10.28,11.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
G06001303650021,7.52,9.86,5.62,5.62,5.62,13.46,7.65,7.39,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
G06001303650022,18.81,18.69,12.72,8.30,10.56,13.66,3.57,3.57,3.57,14.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G06001303860003,12.11,14.95,11.88,11.55,13.29,14.62,16.31,11.35,9.89,12.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
G06001303860004,12.11,14.95,11.88,11.55,13.29,14.62,16.31,11.35,9.89,12.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
G06001303870001,11.96,14.80,11.40,13.14,14.47,11.19,9.74,10.47,2.98,2.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
G06001303870003,11.13,10.57,12.31,13.64,10.36,8.91,7.50,7.50,7.50,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
